## Librerie



In [ ]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
from cleantext import clean
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import plotly.express as px

## Driver


In [ ]:
def start_driver():                                                                       # funzione per inizializzare il driver 
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))           # in questo caso utlizziamo chrome e con la funzione drive manager, viene scaricato il driver della versione corretta
    return driver 

## HTML

In [ ]:
def get_html(driver):                                                                     # funzione per scaricare gli html dei 5 ristoranti 
    
    num_rev = [400, 290, 400, 90, 400]                                                    # numero delle pagine in cui si trovano le ultime recensioni per ogni ristorante

    ID_rist = ["d3741499", "d23807140", "d4318643", "d11785758", "d2062029"]
    
    for i in range(len(ID_rist)):                                                                   # per ogni ristorante

        listaHTML = []


        driver.get(f"https://www.tripadvisor.it/Restaurant_Review-g187785-{ID_rist[i]}-Reviews")                    # apriamo il link con la prima pagina delle recensioni 
        time.sleep(5)
        try:
            driver.find_element(By.ID, "onetrust-accept-btn-handler").click()                                       # prova a premere il pulsante per accettare i cookie
        except:
            pass

        listaHTML.append(driver.page_source)                                                                        # appende la pagina html, scaricata con pagesource, nella lista degli html 
        time.sleep(5)


        for k in range(10, (num_rev[i]-9), 10):
            try:
                driver.get(f"https://www.tripadvisor.it/Restaurant_Review-g187785-{ID_rist[i]}-Reviews-or{k}")      # cicliamo cambiando il numero della pagina all'intero del link per scaricarci il resto delle pagine html
                listaHTML.append(driver.page_source)
                time.sleep(5)
               
            except:
                break


        html_uniti = '\n'.join(listaHTML)                                                                            # uniamo gli html scaricati  

        f = open(f"ristorante_{i+1}.html", 'w',encoding="utf-8")                                                     # salviamo gli html uniti in un unico file 
        f.write(html_uniti)  
        f.close()

    driver.quit()

## Funzione Generale recensioni per singoli ristoranti 


In [ ]:


def get_all_reviews(n):                                                                         # funzione per scaricare tutte le informazioni relative alle recensioni di un ristorante (n)
    
    ID_rist = ["d3741499", "d23807140", "d4318643", "d11785758", "d2062029"]
    
    html = open(f"ristorante_{n}.html", 'r', encoding = "utf-8")                                # apriamo il file del corrispettivo ristorante
    
    all_info_rev = []

    soup = BeautifulSoup(html, "html.parser")                                                   # lo analiziamo con la libreria beautiful soup 
    reviews_boxes = soup.find_all("div", {"class": "review-container"})                         # cerchiamo tramite il corrispetitvo tag il box che contiene le informazioni delle singole recensioni 
    
    for r in reviews_boxes:                                                                     # creiamo un dizionario per ogni recensione del ristorante richiamando le funzioni di acquisizione corrispettive                                                        
        
        dct = {                                                                                 # il dizionario conterrà: id del ristorante, autore della recensione, rating, data della recensione, titolo e testo
            "ID":  ID_rist[n-1],
            "author":  get_author(r),
            "rating":  get_rate(r), 
            "reviewdate":      get_date(r), 
            "title":     get_title(r),
            "text":    get_review(r)}
        
        all_info_rev.append(dct)                                                                # appendiamo i dizionari alla lista dei dizionari, che sarà l'output della funzione

    html.close()
        
    return all_info_rev 

## Funzioni di acquisizione 1


In [ ]:
def get_rate(r):                                                                                # funzione per trovare il rating della recensione r 
    
    if (r.find("span", {"class": "ui_bubble_rating bubble_10"})) is not None:                   # il rating si trova dentro la classe stessa del tag 
        stars = 1                                                                               # dobbiamo imporre delle condizioni sulla classe per trovare il numero di stelle che un utente ha dato nella sua recensione
    elif (r.find("span", {"class": "ui_bubble_rating bubble_20"})) is not None:
        stars = 2
    elif (r.find("span", {"class": "ui_bubble_rating bubble_30"})) is not None:
        stars = 3
    elif (r.find("span", {"class": "ui_bubble_rating bubble_40"})) is not None:
        stars = 4
    elif (r.find("span", {"class": "ui_bubble_rating bubble_50"})) is not None:
        stars = 5
    else:
        stars = ""
        
    return stars

In [ ]:
def get_title(r):                                                                  # funzione per trovare il titolo della recensione r
    value = r.find("span", {"class": "noQuotes"})                                  # troviamo il tag e la classe che contiene il titolo
    if value is None:                                                              # se non c'è il titolo riportiamo una stringa vuota 
        title = ""
    else:
        title = value.text.strip()                                                 # altrimenti prendiamo il testo dentro il tag, lo puliamo dagli spazi inziali e finali e dalle emoji con la libreria cleantext 
        title = clean(title, no_emoji=True)
    return title

In [ ]:
def get_date(r):                                                                   # funzione per trovare la data della recensione
    value = r.find("div", {"class": "prw_rup prw_reviews_stay_date_hsx"})          # troviamo il tag contenitore della data
    if value is None:                                                              # se non c'è riportiamo una stringa vuota
        date = ""
    else: 
        date = value.text.strip()                                                  # altrimenti prendiamo il testo dentro il tag e ci teniamo solo la parte relativa alla data 
        date = date[19:]
    return date

In [ ]:
def get_review(r):                                                                 # funzione per estrarre il testo della recensione 

    review = r.find("div", {"class": "entry"}).text                                # prendiamo il testo facendo riferimento al tag più grande, in modo da prendere anche l'eventuale parte nascosta delle recensioni troppo lunghe
    
    review_piu = review.replace("Più", "")                                         # rimuoviamo la stringa "più" che sarebbe servita per espandere le recensioni più lunghe del limite  
    
    cleaned_review = clean(review_piu, no_emoji=True)                              # ripuliamo la recensione dalle emoji
      
    return cleaned_review

In [ ]:
def get_author(r):                                                                  # funzione per ricavare il nome di chi ha scritto la recensione  
    value = r.find("div", {"class": "info_text pointer_cursor"})                    # troviamo il tag che contiene il nome dell'autore della recensione 
    if value is None:                                                               # se non c'è riportiamo una stringa vuota
        author = ""
    else:
        author = value.text.strip()                                                 # altrimenti prendiamo il testo dentro il tag, lo puliamo e lo mandiamo in output
    return author

## Funzione generale informazioni di tutti e 5 i ristoranti 



In [ ]:
# ESTRAZIONE INFO GENERALI RISTORANTI

def get_info_rist():                                                               # Definiamo la funzione per estrarre le informazioni dai 5 ristoranti 

    all_info_rist = []                                                             # Creiamo una lista vuota
    ID_rist = ["d3741499", "d23807140", "d4318643", "d11785758", "d2062029"]       # Forniamo l'ID dei ristoranti dal link della pagina web 
    
    for i in range(1,6):                                                           # Creiamo un ciclo for per automatizzare l'estrazione delle variabili
        
        html = open(f"ristorante_{i}.html", 'r', encoding = "utf-8")               # Apriamo i vari file html come stringa formattata (f), in modalità lettura (r)
        soup = BeautifulSoup(html, "html.parser")                                  # Utiliziamo la libreria BeautifulSoup, mentre con "html.parser" definiamo la modalità per l'estrazione dei Tag dei ristoranti 
        

        dct = {                                                                    # Il dizionario conterrà: id del ristorante, nome del ristorante, fascia di prezzo, tipo di cucina ed indirizzo
            "ID":  ID_rist[i-1],
            "name":  get_name_rest(soup), 
            "price":      get_price(soup), 
            "typecook":     get_typecook(soup),
            "address":    get_address(soup)}

        all_info_rist.append(dct)                                                  # Appendiamo i dizionari alla lista dei dizionari, che sarà l'output della funzione

        html.close()                                                               # Chiudiamo il file html 
        
    return all_info_rist    

## Funzioni di acquisizione 2


In [ ]:
# Funzione Get Name Restaurant 

def get_name_rest(r):                                                              # Definiamo la funzione per estrarre il nome dei ristoranti 

    name_rest = r.find("h1", {"class": "HjBfq"}).text                              # name_rest -> nome del ristorante (r.find -> primo elemento del box)

    return name_rest

In [ ]:
# Funzione Get Price

def get_price(r):                                                                  # Definiamo la funzione per estrarre la fascia di prezzo del ristorante

    box = r.find("span", {"class": "DsyBj DxyfE"})                                 # box -> scatola che contiene gli span da cercare 
    price = box.find("a", {"class": "dlMOJ"}).text                                 #  price -> prezzo (€€-€€€) 

    return price

In [ ]:
# Funzione Get Type cook

def get_typecook(r):

    box = r.find("span", {"class": "DsyBj DxyfE"})                                 # La funzione cerca uno span nella box
    typecook_list = box.find_all("a", {"class": "dlMOJ"})                          
    
    if typecook_list is None:                                                      # Se la lista typecook_list è vuota (ovvero nessun elemento è stato trovato), viene assegnato una stringa vuota alla variabile typecook
        typecook = ""
    else:
        typecook = []                                                              # Altrimenti, viene inizializzata una lista vuota chiamata typecook
        
        for i in typecook_list:                                                    # Viene iterata la lista typecook_list e per ogni elemento, viene estratto il suo testo e aggiunto alla lista typecook
            i = i.text
            typecook.append(i)
            
    typecook = ", ".join(typecook[1:])                                             # La lista typecook viene quindi convertita in una stringa utilizzando ",".join(typecook[1:])
            
    return typecook

In [ ]:
# Funzione Get Address

def get_address(r):                                                                # Definiamo la funzione per estrarre l'indirizzo del ristorante 

    address = r.find_all("a", {"class": "AYHFM"})                                  # Utilizziamo una find_all sull'oggetto (r) pr trovare tutti gli elemnti (a)
    address = address[1].text                                                      # Poiché l'indirizzo desiderato sembra essere il secondo elemento nella lista, viene estratto il testo dell'elemento address[1] utilizzando il metodo text
    
    return address

## WordCloud

In [ ]:
# WORDCLOUD CON RATING 1-3 E 4-5 DEL RISTORANTE N (da 1 a 5)

def generate_wordcloud(n):                                                         # Definiamo la funzione per generare le WordCloud, prende un parametro n  come input, che rappresenta l'identificatore del ristorante
                                                                                   
    reviews = get_all_reviews(n)                                                   # Richiamiamo la funzione get_all_reviews per ottenere tutte le recensioni del ristorante                                    
    
    
    
    Dataframe = pd.DataFrame.from_dict(reviews)                                    # Le recensioni vengono tutte convertite in un DataFrame pandas
    
    DF_45 = Dataframe[Dataframe["rating"] >= 4]                                    # Dividiamo il DataFrame in due distinti in base al rating delle recensioni
    DF_13 = Dataframe[Dataframe["rating"] < 4]                                     # Rispettivamente di quelle con rating >= a 4 e di quelle con rating < di 4
    
    reviews_45 = " ".join(DF_45["text"].tolist())                                  # Le recensioni dei DataFrame vengono messe in una lista, e poi viene fatto il join per fare una stringa unica
    reviews_13 = " ".join(DF_13["text"].tolist())                               
    
    REG = r"\w+"                                                                   # Questo pattern identificherà parole all'interno di una stringa, escludendo eventuali caratteri di punteggiatura o spazi          
    review_words_45 = re.findall(REG, reviews_45)                                  # Troviamo tutte le parole presenti nella stringa reviews_45 (la stessa cosa vale per reviews_13)
    review_words_13 = re.findall(REG, reviews_13)                                  # Il risultato sarà una lista contenente tutte le parole estratte dalla stringa, che verranno utilizzate per generare il WordCloud

    f = open("stopwords-it.txt", encoding="utf8")                                  # Viene aperto il file "stopwords-it.txt" contenente le parole di che non aggiungono ulteriore significato al testo, che quindi possono essere rimossi.
    stopwords = []                                                                 # Creiamo una lista vuota dove tramite un ciclo for andiamo ad "appendere" ogni parola nel file delle stopwords
    for i in f:
        stopwords.append(i.strip())
    f.close()

    cleaned_review45_words = []                                                    # Vengono create due nuove liste, che contengono le parole delle recensioni che non sono presenti nella lista delle stopwords.
    for k in review_words_45:                                                      # Per fare ciò facciamo un ciclo for nelle liste contenenti le parole delle recensioni, e se la parola trovata non si trova nella lista delle stopwords viene "appesa" nella nuova lista
        if k not in stopwords:
            cleaned_review45_words.append(k)
    unique_review45_words = set(cleaned_review45_words)                            # Le liste di parole vengono convertite in set per ottenere un insieme di parole uniche (sappiamo che il set é una sequenza di elementi unici non ordinati)
    
    cleaned_review13_words = []                                                    # Ripetiamo gli stessi passaggi di prima 
    for k in review_words_13:
        if k not in stopwords:
            cleaned_review13_words.append(k)
    unique_review13_words = set(cleaned_review13_words)

    dict_review45 = {}                                                             
    for j in unique_review45_words:
        dict_review45[j] = cleaned_review45_words.count(j)
                                                                                   # Vengono creati due dizionari, in cui la chiave è una parola unica e il valore è il conteggio di quante volte appare quella parola nelle recensioni corrispondenti
    dict_review13 = {}                                                             
    for j in unique_review13_words:
        dict_review13[j] = cleaned_review13_words.count(j)
        
        
        
        
    wordcloud_df45 = pd.DataFrame.from_dict(dict_review45, orient="index")         # I dizionari vengono convertiti in DataFrame utilizzando il metodo pd.DataFrame.from_dict
    wordcloud_df45.columns = ["count"]                                             # I DataFrame dei dizionari vengono rinominati con la colonna "count"

    wordcloud_df13 = pd.DataFrame.from_dict(dict_review13, orient="index")         # Ripetiamo gli stessi passaggi di prima
    wordcloud_df13.columns = ["count"]

    
    
    wordcloud45 = WordCloud(background_color = "white",                            # Viene creato un oggetto WordCloud per le recensioni con rating 4 o 5 (wordcloud45)
                max_words = 50, width = 600, height = 600)
    
    wordcloud45.generate_from_frequencies(wordcloud_df45.to_dict()["count"])       # Viene generato a partire dalle frequenze delle parole nel DataFrame wordcloud_df45
    
    plt.title(f"Wordcloud delle recensioni con rating 4 o 5 del ristorante {n}")
    plt.imshow(wordcloud45, interpolation="bilinear")                              # Metodo comune per interpolare i valori dei pixel dell"immagine per ottenere una visualizzazione più fluida
    plt.axis("off")
    plt.savefig(f"wordcloud_45_ristorante_{n}.png")
    plt.show()
    
    
    wordcloud13 = WordCloud(background_color = "white",                            # Ripetiamo lo stesso processo per le recensioni con rating da 1 a 3
                max_words = 50, width = 600, height = 600)
    
    wordcloud13.generate_from_frequencies(wordcloud_df13.to_dict()["count"])

    plt.title(f"Wordcloud delle recensioni con rating 1, 2 o 3 del ristorante {n}")
    plt.imshow(wordcloud13, interpolation="bilinear")                                               # Metodo comune per interpolare i valori dei pixel dell"immagine per ottenere una visualizzazione più fluida.
    plt.axis("off")
    plt.savefig(f"wordcloud_13_ristorante_{n}.png")
    plt.show()
    
    count45 = wordcloud_df45.sort_values(['count'], ascending=False)[0:10].rename_axis('parole per rating 4 e 5', axis=1) # I 10 termini più frequenti per ogni rating vengono selezionati dai DataFrame wordcloud_df45 e wordcloud_df13 utilizzando sort_values e vengono rinominati gli assi dei DataFrame.
    count13 = wordcloud_df13.sort_values(['count'], ascending=False)[0:10].rename_axis('parole per rating da 1 a 3', axis=1)
    
    return count45, count13                                                        # Infine, i due DataFrame contenenti i 10 termini più frequenti per ciascun rating vengono restituiti come output della funzione


##Database

In [ ]:


#DATABASE ---------------------------------------------------------------------------------------------------------------------------------------------
def databaseTA():

    db = sqlite3.connect("dbTripAdvisor.db") #Creazione del database e connessione a quest'ultimo


    createTableRistoranti = """CREATE TABLE IF NOT EXISTS ristoranti (

        idristorante STRING NOT NULL PRIMARY KEY,
        nome STRING NOT NULL,
        prezzo FLOAT NOT NULL,
        genereCucina STRING NOT NULL,
        indirizzoRistorante STRING NOT NULL

    );""" #Definizione query di creazione della tabella ristoranti con condizione IF NOT EXISTS

    #genereCucina è il tipo di cucina, ad esempio: italiana, americana, ecc.


    createTableRecensioni = """CREATE TABLE IF NOT EXISTS recensioni (

        idristorante STRING NOT NULL PRIMARY KEY,
        autoreRecensione STRING NOT NULL,
        rating INT NOT NULL,
        titoloRecensione STRING NOT NULL,
        dataRecensione STRING NOT NULL,
        testoRecensione STRING NOT NULL

    );""" #Definizione query di creazione della tabella recensioni con condizione IF NOT EXISTS

    db.execute(createTableRistoranti) #Esecuzione della CREATE TABLE Ristoranti
    db.execute(createTableRecensioni) #Esecuzione della CREATE TABLE Recensioni
    db.commit() #Salvataggio delle operazioni effettuate sul database


    datiRistoranti = get_info_rist() #Richiamo della funzione get_info_rist() che ritorna tutti i dati relativi ai ristoranti
    datiRecensioni = [] #Creazione di una lista vuota che conterrà i dizionari restituiti dalla funzione get_all_reviews()

    for k in range(1,6):
        datiRecensioni.extend(get_all_reviews(k)) #Estensione e creazione dellalista di dizionari contenente tutte le recensioni per ogni ristorante

    nomiColonneDFRistoranti = {'ID':'idristorante',
                            'name':'name',
                            'price':'prezzo',
                            'typecook':'genereCucina',
                            'address':'indirizzoRistorante'} #Creazione dizionario per rinominare gli elementi restituiti dalla funzione get_info_rist()

    nomiColonneDFRecensioni = {'ID':'idristorante',
                               'author':"autoreRecensione",
                               'rating':'rating',
                               'title':'titoloRecensione',
                               'date':'dataRecensione',
                               'text':'testoRecensione'} #Creazione dizionario per rinominare gli elementi restituiti dalla funzione get_all_reviews()

    dfRistoranti = pd.DataFrame(datiRistoranti) #Creazione DatFrame Pandas per immagazzinare i dati sui ristoranti ottenuti in precedenza
    dfRecensioni = pd.DataFrame(datiRecensioni) #Creazione DatFrame Pandas per immagazzinare i dati sulle recensioni ottenuti in precedenza

    dfRistoranti.rename(columns=nomiColonneDFRistoranti, inplace=True) #Rinominazione delle colonne del DataFrame dei ristoranti tramite i dizionari precedentemente definiti
    dfRecensioni.rename(columns=nomiColonneDFRecensioni, inplace=True) #Rinominazione delle colonne del DataFrame delle recensioni tramite i dizionari precedentemente definiti

    print("Dettagli Ristoranti")
    print(dfRistoranti) #Stampa a video del DataFrame contenente i dati dei ristoranti

    print()

    print("Dettagli Recensioni")
    print(dfRecensioni) #Stampa a video del DataFrame contenente i dati delle recensioni

    dfRistoranti.to_sql("ristoranti", db, if_exists='replace', index = False) #Inserimento dei dati contenuti nel DataFrame dei ristoranti nella rispettiva tabella del database tramite il metodo "to_sql", il parametro index indica che gli indici del DataFrame diventeranno quelli delle colonne nel db
    dfRecensioni.to_sql("recensioni", db, if_exists='replace', index = False) #Inserimento dei dati contenuti nel DataFrame delle recensioni nella rispettiva tabella del database tramite il metodo "to_sql", il parametro index indica che gli indici del DataFrame diventeranno quelli delle colonne nel db
    #Il parametro if_exists = 'replace' indica come comportarsi nel caso le tabelle già esistano. In questa caso vengono svuotate e vengono riscritti i dati all'interno. Questo in quanto vogliamo assicurarci che non possano essere introdotte delle ridondanze di dati a seguito di numerose escuzioni del programma

    db.commit() #Salvataggio delle operazioni con il database

    db.close() #Chiusura della connessione con il database

    return dfRistoranti, dfRecensioni #Ritorno dei due DataFrame da parte della funzione



databaseTA() #Chiamata alla funzione database TripAdvisor



##Grafici dei Rating per Ristorante

In [ ]:
def graficiRecensioni(idRistoranteSelect, nomeRistorante, databaseFile):
    
    dbGrafici = sqlite3.connect(databaseFile) #Connessione al dabatase fornito come parametro alla funzione

    selectRecensioniRistorante = f'''SELECT rating 
                                    FROM recensioni
                                    WHERE idristorante == "{idRistoranteSelect}"''' #Query di selezione di tutte le recensioni per un determinato ristorante


    #Acquisizione Ratings
    esecuzioneQueryRatings = dbGrafici.execute(selectRecensioniRistorante) #Esecuzione della query di selezione precedentemente dichiarata
    ratings = esecuzioneQueryRatings.fetchall() #Tramite il metodo fetchall() prende tutte le righe del risultato della query e ritorna una lista di tuple


    #Creazione DataFrame Ratings
    dfRatingGrafico = pd.DataFrame(ratings, columns=["rating"]) #Crea un dataframe rinominando la colonna ottenuta in "rating"
    dfRatingGrafico = dfRatingGrafico.sort_values(by='rating', ascending=False) #Ordinamento dei valori in ordine decrescente




    graficoPxRecensioni = px.pie(dfRatingGrafico, values='rating', names="rating", color_discrete_sequence=px.colors.sequential.RdBu, title=f'<b>Distribuzione dei rating per il ristorante con ID: {idRistoranteSelect} e nome: {nomeRistorante}</b>')
    #Definizione delle caratteristiche del grafico a torta e della palette di colori. Tramite il <b> tag HTML impostazione dei caratteri del titolo in grassetto
    

    #Aggiornamento del layout del grafico
    graficoPxRecensioni.update_layout(
    legend=dict( #Definizione attributi legenda
        title="Ratings", #Titolo della legenda
        title_font=dict(size=14), #Dimensione del titolo
        font=dict(size=25), #Dimensione del font degli elementi nella legenda
        bgcolor="white", #Colore di sfondo
        bordercolor="gray", #Colore del bordo della legenda
        borderwidth=1), #Spessore del bordo della legenda
        title={ #Definizione attributi titolo
        'x': 0.5,  # Posizione orizzontale del titolo nel layout (0-1)
        'y': 0.95,  # Posizione verticale del titolo nel layout (0-1)
        'font': {'size': 20},#, 'weight': 'bold'},  # Impostazione del grassetto per il font del titolo
        'xanchor': 'center',  # Allineamento orizzontale del titolo al centro
        'yanchor': 'top'}  # Allineamento verticale del titolo in alto)
    )

    graficoPxRecensioni.write_html(f'imageGraficoRecensioniRistorante{idRistoranteSelect}.html') #Salvataggio dei grafici sotto forma di file HTML in modo da mantenerne l'interattività

    graficoPxRecensioni.show() #Presentazione a video del grafico

    return 0 #Ritorno 0 della funzione

In [ ]:
dictIDNomiRistoranti = {'d3741499':'Hachi Ristorante Giapponese',
                        'd23807140':'Owap Pizzeria Foria',
                        'd4318643':'Pizza & Sfizi',
                        'd11785758':'Bellini 9',
                        'd2062029':'Tavernetta Colauri'
                        } #Creazione dizionario di corrispondenze tra id e nomi dei ristoranti



In [ ]:
for itr, value in dictIDNomiRistoranti.items(): #Inizializzazione ciclo for iterante contemporaneamente chiavi e valori del dizionario contenente le corrispondenze id e nomi dei ristoranti
    graficiRecensioni(itr, value, "dbTripAdvisor.db") #Esecuzione della funzione di generazione del grafico ratings su ogni ristorante

